In [1]:
r0s = [int(r0si*(10**18)) for r0si in [
  6509056.595038132070616612,
  3308345.021333294051855729,
  34203.974727465737010489,
  1549.689087232146898652,
  954298.037747415499391604
]]

r1s = [int(r1si*(10**18)) for r1si in [
  5348.958422963596672278,
  2707.561367159322649485,
  28.052373793846808553,
  1.265509287402219535,
  782.16393564877776196
]]

r0s, r1s

([6509056595038131770621952,
  3308345021333293932478464,
  34203974727465735028736,
  1549689087232146931712,
  954298037747415521427456],
 [5348958422963596558336,
  2707561367159322968064,
  28052373793846808576,
  1265509287402219520,
  782163935648777764864])

In [2]:
rates = [(r0s[i] / r1s[i]) for i in range(len(r0s))]
rates

[1216.8830041927642,
 1221.8910571930237,
 1219.2898532875054,
 1224.5576564777955,
 1220.0741996060692]

In [3]:
import pandas as pd
from tqdm.notebook import tqdm
df = pd.DataFrame(columns=[ "Start", "End", "Rate i", "Rate j",
                            "startr0", "startr1", "endr0", "endr1",
                            "BorrowToken", "full", "l", "m", "r", "pl"])
df

,Start,End,Rate i,Rate j,startr0,startr1,endr0,endr1,BorrowToken,full,l,m,r,pl


In [4]:
for i in tqdm(range(4)):

    for j in tqdm(range( ( i + 1 ) ,5)):

        if rates[i] > rates[j]:

            skip = 10

            step = int( r0s[i] / skip )

            l0  =  step

            while True:

                if (l0 > r0s[i]):

                    l0  = r0s[i]

                kj = r0s[j] * r1s[j] * (1000**2)

                m1 = r1s[j] - ( kj / ( 1000 * ( ( r0s[j] * 1000 ) + ( l0 * 997 ) ) ) )
                
                ki = r0s[i] * r1s[i] * (1000**2)

                ret1 = ( ( ( ki / 1000 * ( r0s[i] - l0 ) ) - ( r1s[i] * 1000 ) ) / 997 )

                pl = m1 - ret1

                if pl > 0:

                    df = df.append({
                        "Start": i,
                        "End": j,
                        "Rate i": rates[i],
                        "Rate j": rates[j],
                        "startr0": r0s[i],
                        "startr1": r1s[i],
                        "endr0": r0s[j],
                        "endr1": r1s[j],
                        "BorrowToken": "DAI",
                        "full": r0s[i],
                        "l": l0,
                        "m": m1,
                        "ret": ret1,
                        "pl": (m1 - ret1),
                        "ProfitToken": "WETH"
                    }, ignore_index = True )

                if (l0 == r0s[i]):

                    break

                else:

                    l0 += step

            step = int( r1s[j] / skip )

            l1  =  step

            while True:

                if (l1 > r1s[j]):

                    l1  = r1s[j]

                ki = r0s[i] * r1s[i] * (1000**2)

                m0 = r0s[i] - ( ki / ( 1000 * ( ( r1s[i] * 1000 ) + ( l1 * 997 ) ) ) )

                kj = r0s[j] * r1s[j] * (1000**2)

                ret0 = ( ( ( kj / ( 1000 * ( r1s[j] - l1 ) ) ) - ( r0s[j] * 1000 ) ) / 997 )

                pl = m0 - ret0

                if pl > 0:

                    df = df.append({
                        "Start": j,
                        "End": i,
                        "Rate i": rates[i],
                        "Rate j": rates[j],
                        "startr0": r0s[j],
                        "startr1": r1s[j],
                        "endr0": r0s[i],
                        "endr1": r1s[i],
                        "BorrowToken": "WETH",
                        "full": r1s[j],
                        "l": l1,
                        "m": m0,
                        "ret": ret0,
                        "pl": (m0 - ret0),
                        "ProfitToken": "DAI"
                    }, ignore_index = True )

                if (l1 == r1s[j]):

                    break

                else:

                    l1 += step

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [29]:
start_reserve_0 = 6603570092174239050383792
start_reserve_1 = 5152241867963885723178
end_reserve_0 = 1573503599986422669211
end_reserve_1 = 1246920015007834365

In [30]:
fraction_0 = 0.0000011426207020781212
l_0 = 7545375894942212000
m_1 = 5933016134917888
ret_1 = 5904779284986428
( m_1 - ret_1 ) / ( 10**15 )

0.02823684993146

In [24]:
( fraction_0 * start_reserve_0 ), l_0

(7.547340378039528e+18, 7545375894942212000)

In [31]:
initial_ks = start_reserve_0 * start_reserve_1 * 1000, end_reserve_0 * end_reserve_1 * 1000
initial_ks

(34023190306934250428105674086080693330369930976000,
 1962033132509951556020027788399873236015000)

In [32]:
final_ks = ( start_reserve_0 - l_0 ) * ( start_reserve_1 * 1000 + ret_1 * 997 ), ( end_reserve_0 * 1000 + l_0 * 997 ) * ( end_reserve_0 - m_1 )
final_ks

(34023190306934250428107926865454875393914796635072,
 2487741257020989072766546743817572668452725000)

In [33]:
final_ks[0] >= initial_ks[0], final_ks[1] >= initial_ks[1]

(True, True)

In [16]:
final_ks[0] - initial_ks[0]

-3499961745147156376793050559

In [25]:
85481926564761600 / (10**18)

0.0854819265647616

In [41]:
start_reserve_0 = 1573503599986422669211
start_reserve_1 = 1246920015007834365
end_reserve_0 = 3381733268537618195128176
end_reserve_1 = 2621748363220687335971

In [42]:
fraction_1 = 0.007919592780410559
l_1 = 9875098748605470
m_0 = 12699403403684478976
ret_0 = 12598781987567290368
( m_0 - ret_0 ) / ( 10**18 )

0.10062141611718861

In [43]:
initial_ks = start_reserve_0 * start_reserve_1 * 1000, end_reserve_0 * end_reserve_1 * 1000
initial_ks

(1962033132509951556020027788399873236015000,
 8866053661637445612994379313656262412320418896000)

In [44]:
final_ks = ( start_reserve_1 - l_1 ) * ( start_reserve_0 * 1000 + ret_0 * 997 ), ( end_reserve_1 * 1000 + l_1 * 997 ) * ( end_reserve_0 - m_0 )
final_ks

(1962033132509951556187960946902234112854920,
 8866053661637444833473482433395748981200383828000)

In [45]:
final_ks[0] >= initial_ks[0], final_ks[1] >= initial_ks[1]

(True, False)

In [60]:
start_reserve_0 = 1573503599986422669211
start_reserve_1 = 1246920015007834365
end_reserve_0 = 6618361765446170676914026
end_reserve_1 = 5140856343164203936700

fraction_1 = 0.006969405799320131
l_1 = 8690291583883945
m_0 = 11154338226246778880
ret_0 = 11076580448807403520

initial_ks = start_reserve_0 * start_reserve_1 * 1000, end_reserve_0 * end_reserve_1 * 1000
final_ks = ( start_reserve_1 - l_1 ) * ( start_reserve_0 * 1000 + ret_0 * 997 ), ( end_reserve_1 * 1000 + l_1 * 997 ) * ( end_reserve_0 - m_0 )

final_ks[0] >= initial_ks[0], final_ks[1] >= initial_ks[1]

(False, True)

In [61]:
final_ks[0] - initial_ks[0]

-385483296292479479430200

In [62]:
( ( sr1 - l_1 ) * ( sr0 * 1000 + ret_0 * 997 ) ) - ( sr0 * sr1 * 1000 )

NameError: name 'sr1' is not defined

In [ ]:
( ( ( sr0 * sr1 * 1000 ) / ( sr1 - l_1 ) ) - ( sr0 * 1000 ) ) / 997

In [53]:
12699403403684478976 - ( end_reserve_0 - ( initial_ks[1] / ( end_reserve_1 * 1000 + l_1 * 997 ) ) )

0.0

In [54]:
( end_reserve_0 - ( initial_ks[1] / ( end_reserve_1 * 1000 + l_1 * 997 ) ) )

1.269940340368448e+19

In [59]:
( end_reserve_1 * 1000 + l_1 * 997 ) * ( end_reserve_0 - (m_0) ) - (end_reserve_0 * end_reserve_1 * 1000)

-779520896880260513431120035068000